In [1]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd
import json 
import numpy as np
from dotenv import load_dotenv
import os

load_dotenv()
uri = os.getenv("MONGODB_URI")
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

print(client.list_database_names())

db = client.ica_conf

print(db.list_collection_names())

papers_collection = db['papers']
authors_collection = db['authors']
sessions_collection = db['sessions']

['carsApp', 'ica_conf', 'test', 'admin', 'local']
['papers', 'sessions', 'authors']


In [2]:
from fastapi import FastAPI, Query
from pymongo import MongoClient
from typing import List, Optional

app = FastAPI()

In [3]:
PAPERS_JSON = '../data/api/papers.json'
AUTHORS_JSON = '../data/api/authors.json'
SESSIONS_JSON = '../data/api/sessions.json'

In [4]:
papers_data = pd.read_json(PAPERS_JSON).to_dict(orient='records')
authors_data = pd.read_json(AUTHORS_JSON).to_dict(orient='records')
sessions_data = pd.read_json(SESSIONS_JSON).to_dict(orient='records')

In [9]:
papers_df = pd.DataFrame(papers_data)
papers_df.head()

,paper_id,title,paper_type,abstract,number_of_authors,year,session,division,authorships,author_names,session_info
0,2003-0001,Access to the Media Versus Access to Audiences...,Paper,When the issue of speakers' rights of access a...,1,2003,None,None,"[{'position': 0, 'author_name': 'Philip Napoli...",[Philip Napoli],None
1,2003-0002,Accounting Episodes as Communicative Practice ...,Paper,In this paper I describe accounting episodes a...,1,2003,None,None,"[{'position': 0, 'author_name': 'Mariko Kotani...",[Mariko Kotani],None
2,2003-0003,Accounts of Single-fatherhood: A case study,Paper,Abstract\nRelying on single-fathers accounts ...,4,2003,None,None,"[{'position': 0, 'author_name': 'Tara M Emmers...","[Tara M Emmers-Sommer, David Rhea, Laura Tripl...",None
3,2003-0004,A Challenge to the Duel: Socializing Dedicated...,Paper,This paper explores the structural controls av...,1,2003,None,None,"[{'position': 0, 'author_name': 'Chad Tew', 'a...",[Chad Tew],None
4,2003-0005,A chatroom ethnography: Evolution of community...,Paper,"In creating an ethnography about the City, Tex...",1,2003,None,None,"[{'position': 0, 'author_name': 'Audra Diers',...",[Audra Diers],None


In [10]:
non_null_columns = papers_df.columns[papers_df.isnull().sum() == 0]
print("Columns with no null values:", non_null_columns.tolist())

Columns with no null values: ['paper_id', 'title', 'paper_type', 'number_of_authors', 'year']


In [11]:
authors_df = pd.DataFrame(authors_data)
non_null_columns = authors_df.columns[authors_df.isnull().sum() == 0]
print("Columns with no null values:", non_null_columns.tolist())

Columns with no null values: ['author_name', 'attend_count', 'paper_count', 'paper_ids', 'affiliations', 'affiliation_history', 'years_attended']


In [12]:
sessions_df = pd.DataFrame(sessions_data)
non_null_columns = sessions_df.columns[sessions_df.isnull().sum() == 0]
print("Columns with no null values:", non_null_columns.tolist())

Columns with no null values: ['session', 'division']


In [5]:
papers_data[0]

{'paper_id': '2003-0001',
 'title': 'Access to the Media Versus Access to Audiences: The Distinction and its Implications for Media Regulation and Policy',
 'paper_type': 'Paper',
 'abstract': 'When the issue of speakers\' rights of access arises in media regulation and policy contexts, the focus typically is on the concept of speakers\' rights of access "to the media," or "to the press." This right typically is premised on the audience\'s need for access to diverse sources and content. In contrast, in many non-mediated contexts, the concept of speakers\' rights of access frequently is defined in terms of the speaker\'s own First Amendment right of access to audiences. This paper explores the distinctions between these differing interpretations of a speaker\'s access rights and argues that the concept of a speaker\'s right of access to audiences merits a more prominent position in media regulation and policy. This paper then explores the implications of such a shift in perspective for 

In [6]:
authors_data[1000]

{'author_name': 'James H. Watt',
 'attend_count': 6,
 'paper_count': 8,
 'paper_ids': ['2004-0205',
  '2004-0565',
  '2004-0880',
  '2005-0733',
  '2008-1136',
  '2009-0122',
  '2013-0174',
  '2015-0512'],
 'affiliations': ['Rensselaer Polytechnic Institute', 'U of Connecticut'],
 'affiliation_history': 'Rensselaer Polytechnic Institute -> U of Connecticut',
 'years_attended': [2004, 2005, 2008, 2009, 2013, 2015]}

In [7]:
sessions_data[0]

{'session': '"Let\'s Research It All!" New Approaches for Video Games and Their Effects',
 'session_type': 'Paper Session',
 'chair_name': 'Johannes Breuer',
 'chair_affiliation': 'GESIS – Leibniz-Institute for the Social Sciences',
 'division': 'Game Studies',
 'years': [2017],
 'paper_count': 5.0}